# Assignment 3
Student Name: Ziyi Jiang  |  Student ID: 634926886  |  UPI: zjia631

## Section 1 -- Report

###  1. Data Representation

#### Training data
The training data (trg.csv) is stored in to two NumPy ndarry:
1. A ndarry stores all the instances' classes (string).
2. A ndarry stores all the instances' abstractes (string).

Reason:
1. Sine we use the training data to do the testing, we need to keep the class column.
2. We can use the index to find the corresponding abstracte of a class, so the id is not needed.

#### Predicting data
The predicting data (tst.csv) is also stored in to two NumPy ndarrys:
1. A ndarry stores all the instances' ids (string).
2. A ndarry stores all the instances' abstractes (string).

Reason: <br/>
Because tst.csv is the predicting data, we need the ids to record the prediction.

#### In Classifier
The classifier records 6 data parameter:
1. a list of unique class (["A", "B", "E", "V"]) &nbsp;&nbsp;&nbsp; 4. total number of unique words (a dictionary -- {word: number of the word, ...})
2. total number of classes (an integer) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 5. number of a word in a class (a dictionary of dictionary -- {class{word: number of the word in the class, ...}, ...})
3. total number of each class (an integer) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 6. total number of words in a class (an integer)


### 2. Data Preprocessing

#### Preprocessing
1. np.genfromtxt is used to read the CSV file, and then the data is split based on different column.
2. All the data is stored as string in NumPy ndarrys.
3. The individual id and class are stored as one string. For example: "1".
4. The ids and abstractes of tst.csv are stored. For trg.csv, only the classes and abstractes are stored.
5. The abstracte is splited into substrings by " ".
6. The classifier then uses those data to find all needed parameters.

#### Reason
1. The id and class are a single character, so there is no need to split them.
2. For tst.csv we need the id to produce the corresponding prediction, However, we do not need id for trg.csv. We only need to check are the predictions are correct for trg.csv. Hence, the ids of trg.csv is not needed.
3. We need to count the different words in abstractes to calculate the Naive Bayes result, so the abstracte needs to be splited.

### 3. Extension
The Extension used in this report is Complement Naive Bayes.

#### Reason
Here is the data of the trg.csv:
- Number of each class: {'A': 128, 'B': 1602, 'E': 2144, 'V': 126}
- Total word number of each class: {'A': 27529, 'B': 285505, 'E': 379088, 'V': 22700}

We can see that both the class number and word number of B and E are significantly more than A and V. This will causes the skewed data bias which makes the Standard Naive Bayes classifier predict more to B and E. Because the P(A) and P(V) would be much smaller than P(B) and P(E).

However, the Complement Naive Bayes classifier uses the probability of the class divides the probability of a word occured in other class -- P(class) / P(abstracte|other class), which could lower the influence of the skewed data bias and increase the accuracy.

For example, if P(abstracte|class1) is lager than P(abstracte|class2) but P(class1) is much smaller tha P(class2). Then, because P(class1) * P(abstracte|class1) < P(class2) * P(abstracte|class2), the Standard Naive Bayes classifier would predict this abstracte as class2. However, the Complement Naive Bayes classifier would calculates P(class1) / P(abstracte|class2) > P(class2) / P(abstracte|class1) and hence assign this abstracte to class1.

Therefore, for this dataset, since there is significant skewed data bias, Complement Naive Bayes is a good extension.

### 4. Implementation
Due to the 0 probability problem, I use 1 and total unique words number as smoothing parameters:<br/>
&nbsp;&nbsp;&nbsp;&nbsp;
P(class|abstracte) = P(class) $\times$ P(abstracte|class) = P(class) $\times$ $\prod_{}^{}$P(word|class) = P(class) $\times$ $\prod_{}^{}$$\frac{\text{word number } + 1}{\text{total words number in class } + \text{ total unique words number}}$

Due to the underflow problem, the code uses np.log() to record the probabilities (log-recorded probability):<br/>
&nbsp;&nbsp;&nbsp;&nbsp;
log-recorded P(class|abstracte) = log-recorded P(class) + log-recorded P(abstracte|class) <br/>
&nbsp;&nbsp;&nbsp;&nbsp;
= log-recorded P(class) + $\sum_{}^{}$log-recorded P(word|class) <br/> 
&nbsp;&nbsp;&nbsp;&nbsp;
= $\log$$\frac{\text{number of the class}}{\text{total number of classes}}$ + $\sum_{}^{}$$\log$$\frac{\text{word number } + 1}{\text{total words number in class } + \text{ total unique words number}}$

#### Standard Naive Bayes Classifier
The Standard Naive Bayes classifier uses the formula: P(class|abstracte) = P(class) * P(abstracte|class) <br/>
***Detail:*** <br/>
Firstly, I recorded 6 parameter:
1. a list of unique class &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4. total number of unique words
2. total number of classes &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 5. number of a word in a class
3. total number of each class &nbsp;&nbsp;&nbsp; 6. total number of words in a class

Then. I use those parameter to calculate log-recorded P(class|abstracte) of each class:
1. calculate log-recorded P(class) by using np.log(number of the class / total number of classes)
2. calculate log-recorded P(abstracte|class) by sum all log(P(word|class)) of each word in the abstracte
3. use log(P(class)) + log(P(abstracte|class)) to get the log-recorded P(class|abstracte)
4. find the class with the maximum log-recorded P(class|abstracte) -- this would be the predicted class

#### Complement Naive Bayes Classifier
The Complement Naive Bayes classifier uses the formula: P(class|abstracte) = P(class) / P(abstracte|other class) <br/>
***Detail:*** <br/>
Firstly, I recorded the same 6 parameter as in Standard Naive Bayes classifier.<br/>
Then, I calculate P(class|abstracte) of each class:
1. the calculation of log-recorded P(class) is the same as in Standard Naive Bayes classifier
2. calculate log-recorded P(abstracte|other class) by sum all log(P(word|other class)) of each word in the abstracte:
    - a = sum the number of the word in all other classes
    - b = sum the total number of words in all other classes
    - calculate P(word|other class) by using (a + 1) / (b + total number of unique words)
3. use log(P(class)) - log(P(abstracte|other class)) to get the log-recorded P(class|abstracte)
4. find the class with the maximum log-recorded P(class|abstracte) -- this would be the predicted class

### 5. Result

#### Test Method
***Split Method*** <br/>
The train_test_split() funtion uses numpy.random.default_rng().permutation() to generate random indices within the range of training set size. And, those indices are split based on the split size parameter. Then, the function split the training data based on those indices to training set and test set and returns them.<br/>
***Get Accuracy Method*** <br/>
The get_accuracy() funtion convert the result into true and false based on the condition (actual class = predicted class). Then, the function counts how many of the result is true by using a sum() function. And use the count / number of result to get and return the accuracy.

#### Comparision
After ran the two classifiers (Standard Naive Bayes classifier and Complement Naive Bayes classifier) 100 times, and each time uses the same test train split for two classifiers. I got the mean accuracies of two classifier: <br/>
Standard mean: 0.9431875000000001 &nbsp;&nbsp;&nbsp;&nbsp; Standard std: 0.008361696523433503 <br/>
Complement mean: 0.961475 &nbsp;&nbsp;&nbsp;&nbsp; Complement std: 0.00675550701280074 <br/>
We can see that the Complement Naive Bayes classifier's accuracy is about 0.017 more than the Standard Naive Bayes classifier.

For the accuracy of tst.csv, Kaggle shows the Standard Naive Bayes classifier's accuracy is 0.95333 and Complement Naive Bayes classifier's accuracy is 97.

Therefore, for this dataset, the Complement Naive Bayes classifier is more accurate than the Standard Naive Bayes classifier.

## Section 2 -- Code

In [1]:
import numpy as np

In [2]:
def train_test_split(x, y, test_size=0.2):
    x = np.array(x)
    y = np.array(y)
    rng = np.random.default_rng()
    indices = rng.permutation(len(x))
    split_point = int(test_size * len(x))
    test_indices = indices[:split_point]
    train_indices = indices[split_point:]
    return x[train_indices], x[test_indices], y[train_indices], y[test_indices]

In [4]:
def get_accuracy(result):
    result = np.array(result)
    count = sum(result[:, 0] == result[:, 1])
    accuracy = count / len(result)
    return accuracy

### Standard Naive Bayes Classifier

In [5]:
class NaiveBayesClassifier:

    def __init__(self):
        """
        Unique class is stored as self.unique_class.
        Total number of classes is stored as self.total_classes_number.
        Total number of each class is stored in self.class_number_dict.
        Total number of unique words is stored as self.unique_word_number.
        Total number of a word in a class is stored in self.class_word_number_dict.
        otal number of a word in a class is stored in self.class_total_words_dict.
        """
        self.unique_classes = []
        self.total_classes_number = 0
        self.class_number_dict = {}
        self.unique_word_number = 0
        self.class_word_number_dict = {}
        self.class_total_words_dict = {}

    def fit(self, abstracts, classes):
        classes = np.array(classes)
        abstracts = np.array(abstracts)
        # unique class
        self.unique_classes = sorted(set(classes))

        # total number of classes
        self.total_classes_number = len(classes)

        # total number of each class
        c, counts = np.unique(classes, return_counts=True)
        self.class_number_dict = dict(zip(c, counts))

        # total number of unique words
        all_text = ""
        for text in abstracts:
            all_text = all_text + text + " "
        all_text = all_text.split()
        unique_word = set(all_text)
        self.unique_word_number = len(unique_word)

        # total number of a word in a class
        self.class_word_number_dict = {}
        class_text_dict = {}
        for c in self.unique_classes:
            class_text_dict[c] = ""
        for i in range(len(abstracts)):
            c = classes[i]
            text = abstracts[i]
            class_text_dict[c] = class_text_dict[c] + text + " "
        for c in class_text_dict:
            text = np.array(class_text_dict[c].split())
            word, counts = np.unique(text, return_counts=True)
            self.class_word_number_dict[c] = dict(zip(word, counts))

        # total number of words in a class
        self.class_total_words_dict = {}
        for c in class_text_dict:
            text = class_text_dict[c].split()
            self.class_total_words_dict[c] = len(text)

    def predict(self, abstracts, ids):
        result = []
        for i in range(len(ids)):
            pred_id = ids[i]
            pred_abs = abstracts[i].split()
            id_result_prob = []
            for c in self.unique_classes:
                prob_c = np.log(self.class_number_dict[c] / self.total_classes_number)
                prob_x = 0
                word_number_dict = self.class_word_number_dict[c]
                total_words_number = self.class_total_words_dict[c]
                for word in pred_abs:
                    if word in word_number_dict:
                        word_number = word_number_dict[word]
                    else:
                        word_number = 0
                    prob_word = np.log((word_number + 1) / (total_words_number + self.unique_word_number))
                    prob_x = prob_x + prob_word
                prob = prob_c + prob_x
                id_result_prob.append([c, prob])

            id_result_prob = sorted(id_result_prob, key=lambda x: x[1])
            result.append((pred_id, id_result_prob[-1][0]))

        return result

### Navie Bayes Classifier with Extension

#### Complement Naive Bayes

In [6]:
class ComplementNaiveBayesClassifier:

    def __init__(self):
        self.unique_classes = []
        self.total_classes_number = 0
        self.class_number_dict = {}
        self.unique_word_number = 0
        self.class_word_number_dict = {}
        self.class_total_words_dict = {}

    def fit(self, abstracts, classes):
        classes = np.array(classes)
        abstracts = np.array(abstracts)
        
        self.unique_classes = sorted(set(classes))

        self.total_classes_number = len(classes)

        c, counts = np.unique(classes, return_counts=True)
        self.class_number_dict = dict(zip(c, counts))

        all_text = ""
        for text in abstracts:
            all_text = all_text + text + " "
        all_text = all_text.split()
        unique_word = set(all_text)
        self.unique_word_number = len(unique_word)

        self.class_word_number_dict = {}
        class_text_dict = {}
        for c in self.unique_classes:
            class_text_dict[c] = ""
        for i in range(len(abstracts)):
            c = classes[i]
            text = abstracts[i]
            class_text_dict[c] = class_text_dict[c] + text + " "
        for c in class_text_dict:
            text = np.array(class_text_dict[c].split())
            word, counts = np.unique(text, return_counts=True)
            self.class_word_number_dict[c] = dict(zip(word, counts))

        self.class_total_words_dict = {}
        for c in class_text_dict:
            text = class_text_dict[c].split()
            self.class_total_words_dict[c] = len(text)

    def predict(self, abstracts, ids):
        result = []
        for i in range(len(ids)):
            pred_id = ids[i]
            pred_abs = abstracts[i].split()
            id_result_prob = []
            for c in self.unique_classes:
                prob_c = np.log(self.class_number_dict[c] / self.total_classes_number)
                prob_x = 0
                """
                Extension
                """
                not_cs = []
                word_number_in_ncs = []
                total_words_number = 0
                for nc in self.unique_classes:
                    if nc != c:
                        not_cs.append(nc)
                        word_number_in_ncs.append(self.class_word_number_dict[nc])
                        total_words_number += self.class_total_words_dict[nc]  
                for word in pred_abs:
                    word_number = 0
                    for word_number_dict in word_number_in_ncs:
                        if word in word_number_dict:
                            word_number += word_number_dict[word]
                    prob_word = np.log((word_number + 1) / (total_words_number + self.unique_word_number))
                    prob_x = prob_x + prob_word
                prob = prob_c - prob_x
                id_result_prob.append([c, prob])
            id_result_prob = sorted(id_result_prob, key=lambda x: x[1])
            result.append((pred_id, id_result_prob[-1][0]))

        return result

### Test

In [7]:
classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

train_x, test_x, train_y, test_y = train_test_split(train_abstracts, classes)

#### Standard Naive Bayes

In [8]:
nbc =NaiveBayesClassifier()
nbc.fit(train_x, train_y)
result = nbc.predict(test_x, test_y)

nbc_accuracy = get_accuracy(result)
print(nbc_accuracy)

0.95875


#### Complement Naive Bayes

In [9]:
cnbc =ComplementNaiveBayesClassifier()
cnbc.fit(train_x, train_y)
result = cnbc.predict(test_x, test_y)

cnbc_accuracy = get_accuracy(result)
print(cnbc_accuracy)

0.97


#### Comparision

In [10]:
nbc_scores = []
cnbc_scores = []
for i in range(100):
    train_x, test_x, train_y, test_y = train_test_split(train_abstracts, classes)
    
    nbc =NaiveBayesClassifier()
    nbc.fit(train_x, train_y)
    nbc_result = nbc.predict(test_x, test_y)
    nbc_accuracy = get_accuracy(nbc_result)
    nbc_scores.append(nbc_accuracy)
    
    cnbc =ComplementNaiveBayesClassifier()
    cnbc.fit(train_x, train_y)
    cnbc_result = cnbc.predict(test_x, test_y)
    cnbc_accuracy = get_accuracy(cnbc_result)
    cnbc_scores.append(cnbc_accuracy)

print("Standard mean:", np.mean(nbc_scores))
print("Standard std:", np.std(nbc_scores))
print("Complement mean:", np.mean(cnbc_scores))
print("Complement std:", np.std(cnbc_scores))

Standard mean: 0.9431875000000001
Standard std: 0.008361696523433503
Complement mean: 0.961475
Complement std: 0.00675550701280074


### Final Result of tst.csv

#### Standard Naive Bayes

In [11]:
train_classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
train_classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

ids = []
abstracts = []
data = np.genfromtxt("tst.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(0, 1))
ids = data[:, 0]
abstracts = data[:, 1]

nbc = NaiveBayesClassifier()
nbc.fit(train_abstracts, train_classes)
result = nbc.predict(abstracts, ids)

f = open("nbc_result.csv", "a")
f.write("id,class\n")
for r in result:
    line = r[0] + "," + r[1] + "\n"
    f.write(line)
f.close()

#### Complement Naive Bayes

In [12]:
train_classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
train_classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

ids = []
abstracts = []
data = np.genfromtxt("tst.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(0, 1))
ids = data[:, 0]
abstracts = data[:, 1]

cnbc = ComplementNaiveBayesClassifier()
cnbc.fit(train_abstracts, train_classes)
result = cnbc.predict(abstracts, ids)
f = open("cnbc_result.csv", "a")
f.write("id,class\n")
for r in result:
    line = r[0] + "," + r[1] + "\n"
    f.write(line)
f.close()

In [21]:
a = {'A': 128, 'B': 1602, 'E': 2144, 'V': 126}
b = {'A': 27529, 'B': 285505, 'E': 379088, 'V': 22700}

In [25]:
print(sum(a.values()))
print("A:", a["A"] / sum(a.values()))
print("B:", a["B"] / sum(a.values()))
print("E:", a["E"] / sum(a.values()))
print("V:", a["V"] / sum(a.values()))

4000
A: 0.032
B: 0.4005
E: 0.536
V: 0.0315


In [27]:
print(sum(b.values()))
print("A:", b["A"] / sum(b.values()))
print("B:", b["B"] / sum(b.values()))
print("E:", b["E"] / sum(b.values()))
print("V:", b["V"] / sum(b.values()))

714822
A: 0.038511685426581725
B: 0.39940712513045207
E: 0.5303250319659999
V: 0.03175615747696629
